In [1]:
import numpy as np
import pathlib
from PIL import Image
from skimage.io import imread

np.set_printoptions(linewidth=120, precision=3, suppress=True)

In [2]:
directories = ["bricks", "leather", "mosaic", "wood"]
current_dir = pathlib.Path().resolve()

textures_dir = current_dir / 'textures'
samples_dir = current_dir / 'samples'
props_vectors_path = current_dir / 'props_vectors.csv'

sample_size = (128, 128)


def create_sample_dir_for(texture_type, filename):
    p = samples_dir / texture_type / filename
    p.parent.mkdir(parents=True, exist_ok=True)
    return p

def samples_path_for(texture_type: str):
    return samples_dir / texture_type

def read_images_from(directory):
    p = textures_dir / directory
    for f in p.iterdir():
        with Image.open(f) as img:
            yield directory, img

def split_image_into_samples(img: Image, width: int, height: int):
    if img.mode == 'RGBA':
        img = img.convert('RGB')

    img_arr = np.asarray(img)
    iw, ih, _ = img_arr.shape
    samples = []

    splitted_ax_0 = np.split(img_arr, np.arange(width, iw, width))
    if iw % width != 0:
        splitted_ax_0 = splitted_ax_0[:-1]

    hindices = np.arange(height, ih, height)
    for a in splitted_ax_0:
        splitted_ax_1 = np.split(a, hindices, axis=1)
        if ih % height != 0:
            splitted_ax_1 = splitted_ax_1[:-1]
        samples.extend(map(Image.fromarray, splitted_ax_1))
    return samples

def read_grayscale_samples_from(sample_type_dir: pathlib.Path):
    for filepath in sample_type_dir.iterdir():
        yield imread(filepath, as_gray=True)

**Wydzielanie oraz zapis sampli do pliku**

In [3]:
for texture_type_dir in directories:
    for texture_type, img in read_images_from(texture_type_dir):
        sampled = split_image_into_samples(img, *sample_size)
        for i, sample in enumerate(sampled):
            sample.save(create_sample_dir_for(texture_type, f'sample{i}.jpg'))

**Wczytywanie sampli & generowanie wektora do pliku csv**

In [4]:
from skimage.feature import graycomatrix, graycoprops
import itertools
import csv

features = ("dissimilarity", "correlation", "contrast", "energy", "homogeneity", "ASM")
distances = (1, 3, 5)
directions = (0, 45, 90, 135)

with open(props_vectors_path, "w", newline="") as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(("texture_type", "distance", "direction", *features))

    for ttype in directories:
        for sample in read_grayscale_samples_from(samples_path_for(ttype)):
            imarr = (sample * (2**6 - 1)).astype(np.uint8)
            P = graycomatrix(imarr, distances, np.radians(directions), levels=2**6)
            features_array = np.hstack([graycoprops(P, prop).reshape(-1,1) for prop in features])
            for i, (distance, direction) in enumerate(itertools.product(distances, directions)):
                    csvwriter.writerow((ttype, distance, direction, *features_array[i, :]))

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm

df = pd.read_csv(props_vectors_path)

In [7]:
TEST_SIZE = 0.1

train, test = train_test_split(df, test_size=TEST_SIZE)
clf = svm.SVC()
svc = clf.fit(train.loc[:, features], train["texture_type"])

In [8]:
result = clf.predict(test.loc[:, features])

num_errors = (test["texture_type"] != result).sum()
error = num_errors / df.shape[0]
print(
    f"Accuracy: {1-error:.4f} with set size of {df.shape[0]} samples and {TEST_SIZE} test ratio"
)

Accuracy: 0.9864 with set size of 20052 samples and 0.1 test ratio
